In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

# 하이퍼파라미터 설정
input_size = 28  # 입력 시퀀스의 크기 (MNIST 이미지의 가로 길이)
hidden_size = 128  # LSTM의 히든 상태 크기
num_layers = 2  # LSTM의 레이어 개수
num_classes = 10  # 분류할 클래스 개수
batch_size = 100  # 미니배치 크기
num_epochs = 5  # 학습 에포크 수
learning_rate = 0.001  # 학습률

# MNIST 데이터셋 로드 및 전처리
train_dataset = dataset.MNIST(root='./data', train=True, 
                             transform=transforms.ToTensor(), download=True)
test_dataset = dataset.MNIST(root='./data', train=False, 
                             transform=transforms.ToTensor())

# 데이터 로더 생성
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, shuffle=False)

# LSTM 모델 정의
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 생성
model = LSTMNet(input_size, hidden_size, num_layers, num_classes).to(device)

# 손실 함수와 최적화 알고리즘 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 학습
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, input_size, input_size).to(device)
        labels = labels.to(device)

        # 순전파
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')

# 테스트
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {(100 * correct / total)}%')


100%|██████████| 9912422/9912422 [00:00<00:00, 99608308.84it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 86402064.07it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26250949.49it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3524612.17it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Step [100/600], Loss: 0.8110641241073608
Epoch [1/5], Step [200/600], Loss: 0.4865039885044098
Epoch [1/5], Step [300/600], Loss: 0.31776779890060425
Epoch [1/5], Step [400/600], Loss: 0.26064422726631165
Epoch [1/5], Step [500/600], Loss: 0.19294467568397522
Epoch [1/5], Step [600/600], Loss: 0.24102683365345
Epoch [2/5], Step [100/600], Loss: 0.154706671833992
Epoch [2/5], Step [200/600], Loss: 0.11622384935617447
Epoch [2/5], Step [300/600], Loss: 0.1258777678012848
Epoch [2/5], Step [400/600], Loss: 0.22795234620571136
Epoch [2/5], Step [500/600], Loss: 0.1699444204568863
Epoch [2/5], Step [600/600], Loss: 0.06256954371929169
Epoch [3/5], Step [100/600], Loss: 0.04981023818254471
Epoch [3/5], Step [200/600], Loss: 0.10044679045677185
Epoch [3/5], Step [300/600], Loss: 0.08250356465578079
Epoch [3/5], Step [400/600], Loss: 0.0997312143445015
Epoch [3/5], Step [500/600], Loss: 0.07735713571310043
